In [1]:
import cv2
import dlib
import os
import numpy as np
import matplotlib.pyplot as plt
from pyts.image import RecurrencePlot
from joblib import Parallel, delayed

# Extract frames from video for the first 15 seconds
def extract_frames(video_path, max_seconds=5):
    cap = cv2.VideoCapture(video_path)
    frames = []
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    max_frames = max_seconds * fps
    count = 0
    
    while cap.isOpened() and count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
        count += 1
    
    cap.release()
    return frames

# Detect faces in frames
detector = dlib.get_frontal_face_detector()

def detect_faces(frames):
    faces = []
    for frame in frames:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        detected_faces = detector(gray, 1)
        faces.append(detected_faces)
    return faces

# Extract PPG signals from detected faces
def extract_ppg_signals(frames, faces):
    ppg_signals = []
    for i, face_set in enumerate(faces):
        if len(face_set) > 0:
            face = face_set[0]  # Assuming one face per frame
            x, y, w, h = face.left(), face.top(), face.width(), face.height()
            face_region = frames[i][y:y+h, x:x+w]
            mean_color = np.mean(face_region, axis=(0, 1))  # Average color in the face region
            ppg_signals.append(mean_color)
    return np.array(ppg_signals)

# Convert PPG pulse signals to recurrence plot
def convert_to_recurrence_plot(ppg_signals):
    rp = RecurrencePlot()
    recurrence_plot = rp.fit_transform(ppg_signals.reshape(1, -1))
    return recurrence_plot[0]

# Main function to process each video
def process_video(video_path, output_dir):
    frames = extract_frames(video_path)
    # print(f"Extracted {len(frames)} frames from {video_path}")
    
    faces = detect_faces(frames)
    # print(f"Detected faces in {len(faces)} frames from {video_path}")
    
    ppg_signals = extract_ppg_signals(frames, faces)
    # print(f"PPG signals shape for {video_path}: {ppg_signals.shape}")
    
    if ppg_signals.ndim == 2 and ppg_signals.shape[1] == 3:
        recurrence_plot = convert_to_recurrence_plot(ppg_signals[:, 0])  # Using one color channel (e.g., Red)
        
        plt.figure(figsize=(5, 5))  # Ensuring all plots are 5x5 inches
        plt.imshow(recurrence_plot, cmap='binary', origin='lower')
        plt.axis('off')  # Remove axis
        
        plot_path = os.path.join(output_dir, f"{os.path.basename(video_path).replace('.mp4', '_recurrence_plot.png')}")
        plt.savefig(plot_path, bbox_inches='tight', pad_inches=0)
        plt.close()
        
        print(f"Saved recurrence plot for {video_path} to {plot_path}")
    else:
        print(f"PPG signals for {video_path} are not in the expected shape.")

# Function to process videos in separated folders
def process_separated_videos(input_dir, output_dir):
    # fake_input_dir = os.path.join(input_dir, 'FAKE')
    real_input_dir = os.path.join(input_dir, 'REAL')
    
    # fake_output_dir = os.path.join(output_dir, 'FAKE')
    real_output_dir = os.path.join(output_dir, 'REAL')
    
    # os.makedirs(fake_output_dir, exist_ok=True)
    os.makedirs(real_output_dir, exist_ok=True)
    
    def process_single_video(video_path, output_dir):
        process_video(video_path, output_dir)
    
    # Process FAKE videos
    # fake_videos = [os.path.join(fake_input_dir, file) for file in os.listdir(fake_input_dir) if file.endswith('.mp4')]
    # Parallel(n_jobs=-1)(delayed(process_single_video)(video, fake_output_dir) for video in fake_videos)
    
    # Process REAL videos
    real_videos = [os.path.join(real_input_dir, file) for file in os.listdir(real_input_dir) if file.endswith('.mp4')]
    Parallel(n_jobs=-1)(delayed(process_single_video)(video, real_output_dir) for video in real_videos)

# # Run the process
# input_folder_path = 'CELEB'
# output_folder_path = 'celeb_plots'
# process_separated_videos(input_folder_path, output_folder_path)


In [1]:
import os
import numpy as np
import cv2
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet18
from torch import nn
from torch.utils.data import Dataset, DataLoader

# Dataset class for recurrence plots
class RecurrencePlotDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label_dir in ['FAKE', 'REAL']:
            label_path = os.path.join(root_dir, label_dir)
            for file_name in os.listdir(label_path):
                if file_name.endswith('.png'):
                    self.image_paths.append(os.path.join(label_path, file_name))
                    self.labels.append(0 if label_dir == 'FAKE' else 1)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transforms
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset
dataset = RecurrencePlotDataset(root_dir='celeb_plots', transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [8]:
class ResNetClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNetClassifier, self).__init__()
        self.resnet = resnet18(pretrained=True)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)

    def forward(self, x):
        return self.resnet(x)

# Instantiate the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNetClassifier(num_classes=2).to(device)


c:\Users\kuruvadi.intern\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\kuruvadi.intern\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 25

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    val_acc = correct / total
    print(f"Validation Accuracy: {val_acc:.4f}")


Epoch 1/25, Loss: 0.8294
Validation Accuracy: 0.7000
Epoch 2/25, Loss: 0.5141
Validation Accuracy: 0.7481
Epoch 3/25, Loss: 0.4037
Validation Accuracy: 0.6074
Epoch 4/25, Loss: 0.3217
Validation Accuracy: 0.6593
Epoch 5/25, Loss: 0.3128
Validation Accuracy: 0.7259
Epoch 6/25, Loss: 0.2643
Validation Accuracy: 0.8000
Epoch 7/25, Loss: 0.2089
Validation Accuracy: 0.7852
Epoch 8/25, Loss: 0.1892
Validation Accuracy: 0.7926
Epoch 9/25, Loss: 0.1840
Validation Accuracy: 0.7519
Epoch 10/25, Loss: 0.1281
Validation Accuracy: 0.7889
Epoch 11/25, Loss: 0.1095
Validation Accuracy: 0.8074
Epoch 12/25, Loss: 0.0888
Validation Accuracy: 0.8037
Epoch 13/25, Loss: 0.1491
Validation Accuracy: 0.7741
Epoch 14/25, Loss: 0.1161
Validation Accuracy: 0.8000
Epoch 15/25, Loss: 0.0949
Validation Accuracy: 0.8148
Epoch 16/25, Loss: 0.0934
Validation Accuracy: 0.7667
Epoch 17/25, Loss: 0.0614
Validation Accuracy: 0.8296
Epoch 18/25, Loss: 0.0507
Validation Accuracy: 0.8259
Epoch 19/25, Loss: 0.0796
Validation 

In [15]:
def preprocess_video_for_classification(video_path, transform, max_seconds=5):
    frames = extract_frames(video_path, max_seconds)
    faces = detect_faces(frames)
    ppg_signals = extract_ppg_signals(frames, faces)

    if ppg_signals.ndim == 2 and ppg_signals.shape[1] == 3:
        recurrence_plot = convert_to_recurrence_plot(ppg_signals[:, 0])
        recurrence_plot = cv2.resize(recurrence_plot, (224, 224))
        recurrence_plot = np.stack((recurrence_plot,) * 3, axis=-1)  # Make it 3-channel

        if transform:
            recurrence_plot = transform(recurrence_plot)
        return recurrence_plot
    else:
        raise ValueError("PPG signals are not in the expected shape.")

def classify_video(video_path, model, transform):
    model.eval()
    video_tensor = preprocess_video_for_classification(video_path, transform).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(video_tensor)
        _, predicted = torch.max(output, 1)
        return 'REAL' if predicted.item() == 1 else 'FAKE'

# Test the classifier on a new video
test_video_path = r'CELEB\FAKE\id16_id2_0009.mp4'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classification = classify_video(test_video_path, model, transform)
print(f'The video is classified as: {classification}')


The video is classified as: REAL


In [12]:
model_path = r'my_models\celeb_ppg_resnet_lstm_model.pth'
torch.save(model.state_dict(), model_path)

In [9]:
import os
import numpy as np
import cv2
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet18
from torch import nn
from torch.utils.data import Dataset, DataLoader
# Loading the model
class ResNetClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNetClassifier, self).__init__()
        self.resnet = resnet18(pretrained=True)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)

    def forward(self, x):
        return self.resnet(x)
    
model = ResNetClassifier(num_classes=2).to(device)
model.load_state_dict(torch.load(r'my_models\celeb_ppg_resnet_lstm_model.pth'))
model.eval()
print("Model loaded for inference")


Model loaded for inference


In [12]:
def preprocess_video_for_classification(video_path, transform, max_seconds=5):
    frames = extract_frames(video_path, max_seconds)
    faces = detect_faces(frames)
    ppg_signals = extract_ppg_signals(frames, faces)

    if ppg_signals.ndim == 2 and ppg_signals.shape[1] == 3:
        recurrence_plot = convert_to_recurrence_plot(ppg_signals[:, 0])
        recurrence_plot = cv2.resize(recurrence_plot, (224, 224))
        recurrence_plot = np.stack((recurrence_plot,) * 3, axis=-1)  # Make it 3-channel

        if transform:
            recurrence_plot = transform(recurrence_plot)
        return recurrence_plot
    else:
        raise ValueError("PPG signals are not in the expected shape.")

def classify_video(video_path, model, transform):
    model.eval()
    video_tensor = preprocess_video_for_classification(video_path, transform).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(video_tensor)
        probabilities = torch.nn.functional.softmax(output, dim=1)
        confidence, predicted = torch.max(probabilities, 1)
        confidence_score = confidence.item()
        label = 'REAL' if predicted.item() == 1 else 'FAKE'
        return label, confidence_score

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Test the classifier on a new video
test_video_path = r'dfdc_train_part_07\dfdc_train_part_7\kyinkmzqas.mp4'
classification, confidence = classify_video(test_video_path, model, transform)
print(f'The video is classified as: {classification} with confidence score: {confidence:.4f}')


The video is classified as: REAL with confidence score: 0.9994
